In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from io import StringIO

# Logs

In [4]:
def log_message(message):
    with open("./logs/Code_Logs.txt", "a") as f:
        f.write(f"{datetime.now()}: {message}\n")

# Extract

In [5]:
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
page = requests.get(url).content
soup = BeautifulSoup(page, "html.parser")

In [6]:
# tables = soup.find("span", {"class": "mw-headline"}).find_next("table", {"class": "wikitable"})
table = soup.find("div", {"class": "mw-parser-output"}).find_next("table", {"class": "wikitable"})
for header in table:
    th = header.find("thead")
th
# for rows in table:
    

In [7]:
data_table = soup.find("table", {"class": "wikitable"})
data_table

<table class="wikitable sortable mw-collapsible">
<tbody><tr>
<th data-sort-type="number">Rank
</th>
<th>Bank name
</th>
<th>Market cap<br/>(US$ billion)
</th></tr>
<tr>
<td>1
</td>
<td><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><a href="/web/20230908091635/https://en.wikipedia.org/wiki/United_States" title="United States"><img alt="United States" class="mw-file-element" data-file-height="650" data-file-width="1235" decoding="async" height="12" src="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/23px-Flag_of_the_United_States.svg.png" srcset="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/35px-Flag_of_the_United_States.svg.png 1.5x, //web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/46px-Flag_of_the_United_States.svg.png 2x" width="23"/></

In [8]:
table_body = data_table.find("tbody")

rows = table_body.find_all("tr")

# Example: print each row's text content
for row in rows:
    print(row.text.strip())


Rank

Bank name

Market cap(US$ billion)
1

 JPMorgan Chase

432.92
2

 Bank of America

231.52
3

 Industrial and Commercial Bank of China

194.56
4

 Agricultural Bank of China

160.68
5

 HDFC Bank

157.91
6

 Wells Fargo

155.87
7

 HSBC Holdings PLC

148.90
8

 Morgan Stanley

140.83
9

 China Construction Bank

139.82
10

 Bank of China

136.81


In [9]:
for row in rows:
    cells = row.find_all("td")
    cell_values = [cell.text.strip() for cell in cells]
    print(cell_values)

[]
['1', 'JPMorgan Chase', '432.92']
['2', 'Bank of America', '231.52']
['3', 'Industrial and Commercial Bank of China', '194.56']
['4', 'Agricultural Bank of China', '160.68']
['5', 'HDFC Bank', '157.91']
['6', 'Wells Fargo', '155.87']
['7', 'HSBC Holdings PLC', '148.90']
['8', 'Morgan Stanley', '140.83']
['9', 'China Construction Bank', '139.82']
['10', 'Bank of China', '136.81']


In [10]:
data = []
for row in rows:
    cells = row.find_all("td")
    cell_values = [cell.text.strip() for cell in cells]
    if cell_values:  # Only add non-empty rows
        data.append(cell_values)

In [11]:
# Create DataFrame
df = pd.DataFrame(data)

# Export to CSV
df.to_csv("banks_list.csv", index=False, encoding="utf-8")

print("Data successfully saved to 'bank_list.csv'")

Data successfully saved to 'bank_list.csv'


In [14]:
def extract():
    url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
    response = requests.get(url).text
    soup = BeautifulSoup(response, 'lxml')
    table = soup.find('span', string="By market capitalization").find_next('table')
    df = pd.read_html(str(table))[0]
    return df
org_df = extract()

C:\Users\CORECOM\AppData\Local\Temp\ipykernel_10188\3207034511.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


# Transform

In [19]:
def transform(df, csv_path):
    exchange_rate = pd.read_csv(csv_path, index_col=0).to_dict()["rate"]
    print(exchange_rate)
    
    df["Market cap (PKR billion)"] = df["Market cap(US$ billion)"] * exchange_rate["PKR"]
    return df

transform(org_df, './input/currency_rate.csv')

{'pkr': 283.0, 'inr': 86.0, 'gbp': 0.87, 'eur': 0.74}


KeyError: 'Market cap(US$ billion)'